In [ ]:
import pytesseract
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

### Below cell can be used to resize the image

In [ ]:
# down_width = 160
# down_height = 40
# down_points = (down_width, down_height)
# img = cv2.resize(img, down_points, interpolation= cv2.INTER_LINEAR)
# cv2.imshow('image',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
no_filter = []
blur1 = []
thres = []
adap_thres = []
inverted = []
noise_removed = []

In [ ]:
import numpy as np

def noise_removal(image):
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    image = cv2.medianBlur(image, 3)
    return (image)

In [ ]:
#commented the image displaying method reduce the execution time
import os, glob
import cv2
images = [cv2.imread(file,0) for file in glob.glob(r'C:\Users\Jashwanth\Documents\Data mining\Project\data\images/*.jpg')]
for i in images:
#     cv2.imshow('image',img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    data = pytesseract.image_to_string(i, lang='eng', config='--psm 7')
#     print(data)
    no_filter.append(data.rstrip())
    
    
    blur = cv2.GaussianBlur(i, (3,3), 0)
#     cv2.imshow("blurred_image",blur)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    data = pytesseract.image_to_string(blur, lang='eng', config='--psm 7')
#     print(data)
    blur1.append(data.rstrip())
    
    
    
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
#     cv2.imshow("threshold_image",thresh)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    data = pytesseract.image_to_string(thresh, lang='eng', config='--psm 7')
#     print(data)
    thres.append(data.rstrip())
    
    adap_thrs = cv2.adaptiveThreshold(i,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,91,31)
#     cv2.imshow("adap_thresh_image",adap_thrs)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    data = pytesseract.image_to_string(adap_thrs, lang='eng', config='--psm 7')
#     print(data)
    adap_thres.append(data.rstrip())
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    invert = 255 - opening
#     cv2.imshow("inverted_image",invert)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    data = pytesseract.image_to_string(invert, lang='eng', config='--psm 7')
#     print(data)
    inverted.append(data.rstrip())
    
#     no_noise = noise_removal(i)
#     img = noise_removal(no_noise)
#     cv2.imshow("noise_removed_image",no_noise)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#     data = pytesseract.image_to_string(img, lang='eng', config='--psm 7')
#     print(data)
#     noise_removed.append(data)

In [ ]:
len(blur1)

In [ ]:
import pandas as pd
import re

In [ ]:
#appending the data to a dataframe, to comapre the results
nf = pd.DataFrame(no_filter)
nf = nf.replace('\n','', regex=True)
nf = nf[0].map(lambda x: re.sub(r'\W+', '', x))
blr = pd.DataFrame(blur1)
blr = blr.replace('\n','', regex=True)
blr = blr[0].map(lambda x: re.sub(r'\W+', '', x))
thrs = pd.DataFrame(thres)
thrs = thrs.replace('\n','', regex=True)
thrs = thrs[0].map(lambda x: re.sub(r'\W+', '', x))
adap_th= pd.DataFrame(adap_thres)
adap_th = adap_th.replace('\n','', regex=True)
adap_th = adap_th[0].map(lambda x: re.sub(r'\W+', '', x))
inver = pd.DataFrame(inverted)
inver = inver.replace('\n','', regex=True)
inver = inver[0].map(lambda x: re.sub(r'\W+', '', x))

In [ ]:
#displaying the dataframe to understand how the results are stored
nf.tail(10)
blr.head(10)

In [ ]:
#importing the labels into a dataframe
labels = pd.read_csv("labels.txt", sep=" ",names = ['labels'])
l = labels.head(2001)

In [ ]:
# calculating accuracy
#results might vary since the folder contains image with striked words which tesseract couln't detect the text
from sklearn.metrics import accuracy_score
print("accuracy",accuracy_score(l,nf))
print("Blur accuracy",accuracy_score(l,blr))
print("Threshold accuracy",accuracy_score(l,thrs1))
print("AdapThreshold accuracy",accuracy_score(l,adap_th))
print("Inverted accuracy",accuracy_score(l,inver))